In [214]:
import os
import pandas as pd

# Chemin du dossier à analyser
dossier = 'data/raw/'

# Liste pour stocker les infos des fichiers
liste_fichiers = []

# Parcours des fichiers dans le dossier
for fichier in os.listdir(dossier):
    chemin_complet = os.path.join(dossier, fichier)
    if os.path.isfile(chemin_complet):
        nom_sans_ext = os.path.splitext(fichier)[0]
        if len(nom_sans_ext) >= 4 and nom_sans_ext[-4:].isdigit():
            annee = int(nom_sans_ext[-4:])
            nom_restant = nom_sans_ext[:-4]
        else:
            annee = None
            nom_restant = nom_sans_ext
        #récup header
        df_temp = pd.read_csv(chemin_complet, nrows=0, encoding='ISO-8859-1')  # lire juste les entêtes
        headers = list(df_temp.columns)
      

        liste_fichiers.append({
            'fichier': fichier,
            'annee': annee,
            'fichier_categorie': nom_restant,
            'path':chemin_complet,
            'header': headers
            
        })

# Création du DataFrame
df = pd.DataFrame(liste_fichiers)

# Affichage du résultat
print(df)

                       fichier   annee       fichier_categorie  \
0   Accident_auto_combined.csv     NaN  Accident_auto_combined   
1              caract-2023.csv  2023.0                 caract-   
2    caracteristiques-2017.csv  2017.0       caracteristiques-   
3    caracteristiques-2018.csv  2018.0       caracteristiques-   
4    caracteristiques-2019.csv  2019.0       caracteristiques-   
5    caracteristiques-2020.csv  2020.0       caracteristiques-   
6    caracteristiques_2012.csv  2012.0       caracteristiques_   
7    caracteristiques_2013.csv  2013.0       caracteristiques_   
8    caracteristiques_2014.csv  2014.0       caracteristiques_   
9    caracteristiques_2015.csv  2015.0       caracteristiques_   
10   caracteristiques_2016.csv  2016.0       caracteristiques_   
11    carcteristiques-2021.csv  2021.0        carcteristiques-   
12    carcteristiques-2022.csv  2022.0        carcteristiques-   
13              lieux-2017.csv  2017.0                  lieux-   
14        

In [215]:
#homogénéisation de la colonne fichier_categorie
df['fichier_categorie'] = df['fichier_categorie'].replace('lieux-', 'lieux_')
df['fichier_categorie'] = df['fichier_categorie'].replace('usagers-', 'usagers_')
df['fichier_categorie'] = df['fichier_categorie'].replace('vehicules-', 'vehicules_')
df.loc[df['fichier_categorie'].isin(['caract-', 'caracteristiques-', 'carcteristiques-']), 'fichier_categorie'] = 'caracteristiques_'
df.loc[df['fichier_categorie'].isin(['lieux-', 'caracteristiques-', 'carcteristiques']), 'fichier_categorie'] = 'caracteristiques_'
print(df['fichier_categorie'].drop_duplicates())

0     Accident_auto_combined
1          caracteristiques_
13                    lieux_
25                  usagers_
37                vehicules_
Name: fichier_categorie, dtype: object


In [216]:
print(df['header'].drop_duplicates())

0     [Num_Acc, id_usager, id_vehicule, num_veh, pla...
1     [Num_Acc;"jour";"mois";"an";"hrmn";"lum";"dep"...
2     [Num_Acc, an, mois, jour, hrmn, lum, agg, int,...
12    [Accident_Id;"jour";"mois";"an";"hrmn";"lum";"...
13    [Num_Acc, catr, voie, v1, v2, circ, nbv, pr, p...
15    [Num_Acc;"catr";"voie";"v1";"v2";"circ";"nbv";...
25    [Num_Acc, place, catu, grav, sexe, trajet, sec...
27    [Num_Acc;"id_vehicule";"num_veh";"place";"catu...
29    [Num_Acc;"id_usager";"id_vehicule";"num_veh";"...
37    [Num_Acc, senc, catv, occutc, obs, obsm, choc,...
39    [Num_Acc;"id_vehicule";"num_veh";"senc";"catv"...
Name: header, dtype: object


In [217]:
#enregistrement du listing de fichier + header
# 
df.to_csv("data/raw/listingfiles.csv", index=False,)

In [218]:
def get_data(Annee, Cat, df):
    ligne = df.loc[(df["annee"] == Annee) & (df["fichier_categorie"] == Cat), "path"]
    header = df.loc[(df["annee"] == Annee) & (df["fichier_categorie"] == Cat), "header"]
    
    if not ligne.empty:
        chemin = ligne.iloc[0]
        header_val = header.iloc[0]
        separa = ';' if any(';' in col for col in header_val) else ','
        
        try:
            # Ajouter low_memory=False pour éviter les warnings DtypeWarning
            return pd.read_csv(chemin, sep=separa, low_memory=False)
        except FileNotFoundError:
            print(f"❌ Fichier non trouvé : {chemin}")
            return pd.DataFrame()  # DataFrame vide plutôt que None
        except Exception as e:
            print(f"❌ Erreur lors de la lecture de {chemin}: {e}")
            return pd.DataFrame()
    else:
        print(f"❌ Aucune configuration trouvée pour année={Annee}, catégorie={Cat}")
        return pd.DataFrame()  # DataFrame vide plutôt que None

In [219]:
def get_annee_dispo(annees_list, annee_min, annee_max):
    """
    trie une liste d'année en tenant compte d'un mini et d'un maxi
    ex: print(get_annee_dispo(df['annee'],'2020','2021'))
    """
    import pandas as pd

    def safe_int(value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return None

    annee_min = safe_int(annee_min)
    annee_max = safe_int(annee_max)

    # Nettoyage ici, sur la série brute
    annees = pd.to_numeric(annees_list, errors='coerce').dropna().astype(int)

    if annee_min is not None and annee_max is not None:
        annees = annees[annees.between(annee_min, annee_max)]
    elif annee_min is not None:
        annees = annees[annees >= annee_min]
    elif annee_max is not None:
        annees = annees[annees <= annee_max]

    return annees.drop_duplicates().sort_values(ascending=False).reset_index(drop=True)

# Utilisation
#print(list_annee)


In [220]:
all_accidents = [] 
annee_dispo=get_annee_dispo(df['annee'],'2019','2023')
for annee in annee_dispo:
    # On ouvre les fichiers
    print(f"\nTraitement des fichiers pour l'année {annee}...")
    caract = get_data(annee, 'caracteristiques_', df)
    lieux = get_data(annee, 'lieux_', df)
    usagers= get_data(annee, 'usagers_', df)
    vehicules= get_data(annee, 'vehicules_', df)
    # homogenisation colonne Num_acc
    if 'Accident_Id' in caract.columns:
        caract = caract.rename(columns={'Accident_Id': 'Num_Acc'})
    try:
        # merge des différentes tables avec gestion des erreurs 
        merged= (usagers
            .merge(lieux, on='Num_Acc')
            .merge(vehicules, on=['Num_Acc', 'id_vehicule', 'num_veh'], how='inner')
            .merge(caract, on='Num_Acc')
        )
        all_accidents.append(merged)
        print(f"Fusion réussie pour l'année {annee}")
    except KeyError as e:
        print(f"Erreur : colonne manquante {e}")
    except Exception as e:
        print(f"Erreur lors de la fusion : {e}")  
    print(f"accident{annee}")
#On concatène (à voir si encore utile)    
combined_data = pd.concat(all_accidents, ignore_index=True)
#enregistrement du fichier final
combined_data.to_csv('data/raw/Accident_auto_combined.csv', index=False, encoding='utf-8')
print(f"Combined export: {len(combined_data)} total records across all years")
#pd.DataFrame([all_accidents]).to_csv('Accident_auto.csv', index=False, encoding='utf-8')  # ou 'ISO-8859-1' si besoin


Traitement des fichiers pour l'année 2023...
Fusion réussie pour l'année 2023
accident2023

Traitement des fichiers pour l'année 2022...
Fusion réussie pour l'année 2022
accident2022

Traitement des fichiers pour l'année 2021...
Fusion réussie pour l'année 2021
accident2021

Traitement des fichiers pour l'année 2020...
Fusion réussie pour l'année 2020
accident2020

Traitement des fichiers pour l'année 2019...
Fusion réussie pour l'année 2019
accident2019
Combined export: 657865 total records across all years
